In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark
import org.apache.spark.sql._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.sql.Column
import org.apache.spark.sql.types.{DataType, DateType, TimestampType}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.DataFrameNaFunctions
import org.apache.spark.sql.DataFrameStatFunctions
import org.apache.spark.sql.sources._


var spark = SparkSession
.builder()
.appName("Spark HW 1")
.config("spark.master", "local")
.getOrCreate();

def sc = spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                           // Added automatically on importing Spark

import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}

import org.apache.spark.SparkContext

import org.apache.spark.SparkConf

import org.apache.spark.sql.Column

import org.apache.spark.sql.types.{DataType, DateType, TimestampType}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.DataFrameNaFunctions

import org.apache.spark.sql.DataFrameStatFunctions

import org.apache.spark.sql.sources._



spark: SparkSession = org.apache.spark.sql.SparkSession@248073ec
defined function sc

In [11]:
/*******  Task 1  ********/

val df = spark.read.
option("header", true).
option("inferSchema",true).
csv("./SampledData.csv").
withColumn("date", to_date(col("timestamp")))

val window_func = Window.partitionBy("device_id").orderBy("timestamp")

df.select("device_id", "country_code").
groupBy(lower(col("country_code")).alias("country")).agg(countDistinct("device_id").alias("active_users in US")).
filter(lower(col("country_code")) === "us").
withColumn("perc_of_total_users", (col("active_users in US")/df.select("device_id").distinct.count)*100).
show(100)

df.select("device_id", "timestamp", "event_type", "country_code").
filter(col("event_type") === "editor_done_click").
filter(lower(col("country_code")) === "us").
withColumn("editor_done_click", lead("timestamp", 1) over window_func).
groupBy(lower(col("country_code")).alias("country")).agg(countDistinct("device_id").alias("editor_done_click >= 3")).
filter(col("editor_done_click >= 3") >= 3).
withColumn("perc_of_US_users", col("editor_done_click >= 3")/(df.select("device_id").
                                                         filter(lower(col("country_code"))==="us")).distinct.count*100).
orderBy(desc("perc_of_US_users")).show(100, false)

+-------+------------------+-------------------+
|country|active_users in US|perc_of_total_users|
+-------+------------------+-------------------+
|     us|                 6|  6.122448979591836|
+-------+------------------+-------------------+

+-------+----------------------+-----------------+
|country|editor_done_click >= 3|perc_of_US_users |
+-------+----------------------+-----------------+
|us     |4                     |66.66666666666666|
+-------+----------------------+-----------------+



df: DataFrame = [device_id: string, country_code: string ... 13 more fields]
window_func: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@2b3aefad

In [3]:
/*******  Task 2  ********/

val df = spark.read.
option("header", true).
option("inferSchema",true).
csv("./photo_uploadTest.csv")

df.select("device_id", "also_sharedT").
groupBy("also_sharedT").
agg(countDistinct("device_id").alias("users instagram shared")).
filter(col("also_sharedT") === "instagram").
show(100, false)

+------------+----------------------+
|also_sharedT|users instagram shared|
+------------+----------------------+
|instagram   |28                    |
+------------+----------------------+



df: DataFrame = [Accept-Encoding: string, advertising_id: string ... 67 more fields]

In [4]:
/*******  Task 3  ********/

val df = spark.read.
option("header", true).
option("inferSchema",true).
csv("./SampledData.csv").
withColumn("date", to_date(col("timestamp"))).distinct

val installs = spark.read.
option("header", true).
option("inferSchema",true).
csv("./installsTest.csv").
withColumn("install_date", to_date(col("timestamp_ins")))

var min = df.agg(min(col("date"))).take(1)(0)(0).toString()
var max = df.agg(max(col("date"))).take(1)(0)(0).toString()

df: Dataset[Row] = [device_id: string, country_code: string ... 13 more fields]
installs: DataFrame = [device_id: string, timestamp_ins: timestamp ... 14 more fields]
min: String = "2018-02-02"
max: String = "2018-02-18"

In [6]:
df.select("device_id").as("a").
join(installs.as("b"), col("a.device_id") === col("b.device_id"),"left").
withColumn("userType",
           when(col("b.device_id").isNull,lit("Old"))
           when(col("b.device_id").isNotNull && 
                to_date(col("timestamp_ins")).between(min,max),lit("New"))
           otherwise(lit("Old"))).
groupBy("userType").
agg(countDistinct("a.device_id").as("cnt")).
show(100,false)

+--------+---+
|userType|cnt|
+--------+---+
|Old     |90 |
|New     |8  |
+--------+---+



In [7]:
/*******  Task 4  ********/

val df = spark.read.
option("header", true).
option("inferSchema",true).
csv("./sticker_save.csv").
withColumn("date", to_date(col("timestamp"))).distinct

df.printSchema


root
 |-- Accept-Encoding: string (nullable = true)
 |-- advertising_id: string (nullable = true)
 |-- app: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- debug: boolean (nullable = true)
 |-- device_id: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- event_type: string (nullable = true)
 |-- is_following: boolean (nullable = true)
 |-- language_code: string (nullable = true)
 |-- market: string (nullable = true)
 |-- method: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- sid: string (nullable = true)
 |-- source: string (nullable = true)
 |-- sticker_id: long (nullable = true)
 |-- timezone: string (nullable = true)
 |-- user_id: long (nullable = true)
 |-- v: string (nullable = true)
 |-- version: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- date: date (nullable = true)



df: Dataset[Row] = [Accept-Encoding: string, advertising_id: string ... 22 more fields]

In [2]:
/*******  Task 5  ********/
/** Face for overall Beautify ***/

val df = spark.read.
option("header", true).
option("inferSchema",true).
csv("./SampledData.csv").
withColumn("date", to_date(col("timestamp"))).distinct

val beautify = spark.read.
option("header", true).
option("inferSchema",true).
csv("./apply_beautify.csv").
withColumn("date", to_date(col("timestamp"))).distinct

// df.select("event_type").orderBy("event_type").distinct.show(400, false)

df: Dataset[Row] = [device_id: string, country_code: string ... 13 more fields]
beautify: Dataset[Row] = [Accept-Encoding: string, advertising_id: string ... 21 more fields]

In [3]:
// Opens with face photos

df.select("device_id", "event_type").distinct.as("a").
join(beautify.as("b"), col("a.device_id") === col("b.device_id"),"left").
filter(col("a.event_type") === "photo_open").
groupBy("a.event_type").
agg(count("a.device_id").as("distinct face photo opens")).
show(100, false)

+----------+-------------------------+
|event_type|distinct face photo opens|
+----------+-------------------------+
|photo_open|15                       |
+----------+-------------------------+



In [4]:
// count how many times face tool applied

beautify.select("device_id", "tool").
filter(col("tool") === "face_fix").
groupBy("tool").
agg(count("device_id").as("# of times used")).show

+--------+---------------+
|    tool|# of times used|
+--------+---------------+
|face_fix|           1491|
+--------+---------------+



In [ ]:
// change (%) in subscribed users' shares 

// How can we know if a user is subscribed or not?

In [ ]:
/*******  Task 6  ********/
/** For subscribed users only ***/
// Beautify Open -> Apply (conversion)
// Stickiness
// Engagement 
// Most used tools' combinations
// Try -> Apply per tool comparison

val df = spark.read.
option("header", true).
option("inferSchema",true).
csv("./SampledData.csv").
withColumn("date", to_date(col("timestamp"))).distinct

val beautify = spark.read.
option("header", true).
option("inferSchema",true).
csv("./apply_beautify.csv").
withColumn("date", to_date(col("timestamp"))).distinct

In [24]:
// Beautify Open -> Apply (conversion)
df.select("device_id", "event_type").distinct.as("a").
join(beautify.as("b"), col("a.device_id") === col("b.device_id"), "left").
filter(col("a.event_type") === "editor_open").
groupBy("a.event_type").
agg(count("a.device_id").as("editor_open -> apply_beautify")).
show(100, false)

+-----------+-----------------------------+
|event_type |editor_open -> apply_beautify|
+-----------+-----------------------------+
|editor_open|79                           |
+-----------+-----------------------------+



In [ ]:
/*******  Task 7  ********/
/** Face Tool (before/after Multiface support) ***/
// Face Open -> Offer View -> Trial -> Paid (conversions)
// Frequency of applies

